# Pres Individual netCDF Files for Each Variable (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta

# pres

In [3]:
os.chdir('d:/NCEP_R2')
ds_pres= xr.open_dataset('pres.sfc.mon.mean.nc') # ds_precip

### cut new clim_prate
- prate
- clim_ds
- anom_ds
- std_ds

In [4]:
new_clim = ds_pres.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    pres     (time, lat, lon) float32 ...

In [6]:
# unit : Pascals -> hPa
pa_to_hpa = 0.01 # [hPa]
pres = new_clim.pres*pa_to_hpa

lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time

#### ◆ pres nc file (monthly mean)

In [9]:
pres_arr = np.array(pres)

pres_ds = xr.Dataset(
    data_vars = dict(
        pres = (['time','lat','lon'],pres_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

pres_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
pres_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
pres_ds.pres.attrs ={'units':'hPa','long_name':'Monthly Mean of Forecast of Surface Pressure'}


In [10]:
# pres_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_mon_mean_new.nc')

#### ◆ pres clim nc file

In [11]:
clim_sel = new_clim['pres'].groupby('time.month').mean('time')

clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        new_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.new_clim.attrs ={'units':'hPa','long_name':'Average Long Term Mean Monthly Rate of Pressure at Surface'}

In [12]:
# clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_clim_new.nc')

#### ◆ pres anom nc file

In [14]:
anom_lst = []
for i in range(1,13):
    each_mon = new_clim['pres'].sel(time=new_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # new_clim['pres'].groupby('time.month').mean('time')
    clim_mon = clim_ds['new_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        new_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.new_anom.attrs ={'units':'mm/month','long_name':'The Monthly Anomaly Rate of Forecasted Surface Pressure'} 
anom_ds


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lat: 94, lon: 192)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    new_anom  (time, lat, lon) float32 -269.7 -269.3 -260.3 ... -362.5 -358.9

In [15]:
# anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_anom_new.nc')

#### ◆ pres std nc file

In [16]:
## std
std_sel=new_clim.pres.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        new_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.new_std.attrs ={'units':'mm/month','long_name':'Standard Deviation Rate of Forecasted Surface Pressure'}

In [18]:
os.getcwd()
# std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NECP_pres_sfc_std_new.nc')

### cut old clim_prate
- prate
- clim_ds
- anom_ds
- std_ds

In [20]:
old_clim = ds_pres.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    pres     (time, lat, lon) float32 ...

In [21]:
# unit : Pascals -> hPa
pa_to_hpa = 0.01 # [hPa]
pres = old_clim.pres*pa_to_hpa

lon = old_clim.lon
lat = old_clim.lat
time = old_clim.time

#### ◆ pres nc file (monthly mean)

In [22]:
pres_arr = np.array(pres)

pres_ds = xr.Dataset(
    data_vars = dict(
        pres = (['time','lat','lon'],pres_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

pres_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
pres_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
pres_ds.pres.attrs ={'units':'hPa','long_name':'Monthly Mean of Forecast of Surface Pressure'}


In [23]:
# pres_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_mon_mean_old.nc')

#### ◆ pres clim nc file

In [24]:
clim_sel = old_clim['pres'].groupby('time.month').mean('time')

clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        old_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.old_clim.attrs ={'units':'hPa','long_name':'Average Long Term Mean Monthly Rate of Pressure at Surface'}

In [25]:
# clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_clim_old.nc')

#### ◆ pres anom nc file

In [26]:
anom_lst = []
for i in range(1,13):
    each_mon = old_clim['pres'].sel(time=old_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # old_clim['pres'].groupby('time.month').mean('time')
    clim_mon = clim_ds['old_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        new_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
anom_ds.lon.attrs = old_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.new_anom.attrs ={'units':'mm/month','long_name':'The Monthly Anomaly Rate of Forecasted Surface Pressure'} 
anom_ds


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lat: 94, lon: 192)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    new_anom  (time, lat, lon) float32 -997.0 -996.3 -1.006e+03 ... 477.7 474.7

In [27]:
# anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NCEP_pres_sfc_anom_old.nc')

#### ◆ pres std nc file

In [28]:
## std
std_sel=old_clim.pres.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        new_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.new_std.attrs ={'units':'mm/month','long_name':'Standard Deviation Rate of Forecasted Surface Pressure'}

In [29]:
os.getcwd()
# std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/pres/NECP_pres_sfc_std_old.nc')